In [1]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
from google.colab  import files
from zipfile import ZipFile
import pandas as pd
import glob
import random
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from keras.layers import Dense , BatchNormalization, Dropout
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from keras.layers import LSTM 
from keras.layers import Bidirectional, GlobalMaxPool1D
from sklearn.model_selection import KFold,cross_val_score

Using TensorFlow backend.


In [0]:
!pip install -q kaggle

In [8]:
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [10]:
!kaggle datasets download -d nnair25/Alcoholics

 97% 279M/287M [00:02<00:00, 89.5MB/s]
100% 287M/287M [00:02<00:00, 119MB/s] 


In [0]:
file_name = "Alcoholics.zip"

In [12]:
with ZipFile(file_name, 'r') as zip: 
    zip.printdir() 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

File Name                                             Modified             Size
SMNI_CMI_TEST/Data1.csv                        2019-09-20 23:06:22       929723
SMNI_CMI_TEST/Data10.csv                       2019-09-20 23:06:24       930179
SMNI_CMI_TEST/Data100.csv                      2019-09-20 23:06:24       928464
SMNI_CMI_TEST/Data101.csv                      2019-09-20 23:06:26      1045391
SMNI_CMI_TEST/Data102.csv                      2019-09-20 23:06:28      1043753
SMNI_CMI_TEST/Data103.csv                      2019-09-20 23:06:28       965222
SMNI_CMI_TEST/Data104.csv                      2019-09-20 23:06:30      1047920
SMNI_CMI_TEST/Data105.csv                      2019-09-20 23:06:30       965789
SMNI_CMI_TEST/Data106.csv                      2019-09-20 23:06:30      1043848
SMNI_CMI_TEST/Data107.csv                      2019-09-20 23:06:32      1049820
SMNI_CMI_TEST/Data108.csv                      2019-09-20 23:06:32       964327
SMNI_CMI_TEST/Data109.csv               

In [0]:
path =r'/content/SMNI_CMI_TEST'

filenames = glob.glob(path + "/*.csv")

dfs = []

for filename in filenames:

    dfs.append(pd.read_csv(filename))



Test_data = pd.concat(dfs, ignore_index=True)



In [0]:
test = pd.DataFrame(Test_data)

In [0]:

path =r'/content/SMNI_CMI_TRAIN'

filenames = glob.glob(path + "/*.csv")

dfs = []

for filename in filenames:

    dfs.append(pd.read_csv(filename))



Train_data = pd.concat(dfs, ignore_index=True)

train = pd.DataFrame(Train_data)


In [120]:
print(train)


         Unnamed: 0  trial number  ...         name      time
0                 5            14  ...  co2a0000375  0.000000
1                 6            14  ...  co2a0000375  0.003906
2                 7            14  ...  co2a0000375  0.007812
3                 8            14  ...  co2a0000375  0.011719
4                 9            14  ...  co2a0000375  0.015625
...             ...           ...  ...          ...       ...
7667707       16447            37  ...  co2a0000371  0.980469
7667708       16448            37  ...  co2a0000371  0.984375
7667709       16449            37  ...  co2a0000371  0.988281
7667710       16450            37  ...  co2a0000371  0.992188
7667711       16451            37  ...  co2a0000371  0.996094

[7667712 rows x 10 columns]


In [121]:
print(test)

         Unnamed: 0  trial number  ...         name      time
0                 5            39  ...  co2a0000375  0.000000
1                 6            39  ...  co2a0000375  0.003906
2                 7            39  ...  co2a0000375  0.007812
3                 8            39  ...  co2a0000375  0.011719
4                 9            39  ...  co2a0000375  0.015625
...             ...           ...  ...          ...       ...
7864315       16447            67  ...  co2a0000371  0.980469
7864316       16448            67  ...  co2a0000371  0.984375
7864317       16449            67  ...  co2a0000371  0.988281
7864318       16450            67  ...  co2a0000371  0.992188
7864319       16451            67  ...  co2a0000371  0.996094

[7864320 rows x 10 columns]


In [122]:
EEG_data = train.drop(['Unnamed: 0'], axis=1)
print(EEG_data)

         trial number sensor position  ...         name      time
0                  14             FP1  ...  co2a0000375  0.000000
1                  14             FP1  ...  co2a0000375  0.003906
2                  14             FP1  ...  co2a0000375  0.007812
3                  14             FP1  ...  co2a0000375  0.011719
4                  14             FP1  ...  co2a0000375  0.015625
...               ...             ...  ...          ...       ...
7667707            37               Y  ...  co2a0000371  0.980469
7667708            37               Y  ...  co2a0000371  0.984375
7667709            37               Y  ...  co2a0000371  0.988281
7667710            37               Y  ...  co2a0000371  0.992188
7667711            37               Y  ...  co2a0000371  0.996094

[7667712 rows x 9 columns]


In [0]:
# EEG_data.loc[EEG_data['sensor position'] == 'AF1', 'sensor position'] = 'AF3'
# EEG_data.loc[EEG_data['sensor position'] == 'AF2', 'sensor position'] = 'AF4'
# EEG_data.loc[EEG_data['sensor position'] == 'PO1', 'sensor position'] = 'PO3'
# EEG_data.loc[EEG_data['sensor position'] == 'PO2', 'sensor position'] = 'PO4'
# ## remove rows with undefined positions
# EEG_data = EEG_data[(EEG_data['sensor position'] != 'X') & (EEG_data['sensor position'] != 'Y') & (EEG_data['sensor position'] != 'nd')]

In [0]:
EEG_data = EEG_data.reset_index()

In [124]:
EEG_data[EEG_data['sample num'] == 0]

,index,trial number,sensor position,sample num,sensor value,subject identifier,matching condition,channel,name,time
0,0,14,FP1,0,9.430,a,S1 obj,0,co2a0000375,0.0
256,256,14,FP2,0,10.071,a,S1 obj,1,co2a0000375,0.0
512,512,14,F7,0,12.990,a,S1 obj,2,co2a0000375,0.0
768,768,14,F8,0,13.357,a,S1 obj,3,co2a0000375,0.0
1024,1024,14,AF1,0,8.881,a,S1 obj,4,co2a0000375,0.0
...,...,...,...,...,...,...,...,...,...,...
7666432,7666432,37,P2,0,-3.682,a,S2 match,59,co2a0000371,0.0
7666688,7666688,37,P1,0,-2.319,a,S2 match,60,co2a0000371,0.0
7666944,7666944,37,CPZ,0,-1.292,a,S2 match,61,co2a0000371,0.0
7667200,7667200,37,nd,0,3.204,a,S2 match,62,co2a0000371,0.0


In [125]:
senPos = EEG_data['sensor position'].unique()
print(senPos)

['FP1' 'FP2' 'F7' 'F8' 'AF1' 'AF2' 'FZ' 'F4' 'F3' 'FC6' 'FC5' 'FC2' 'FC1'
 'T8' 'T7' 'CZ' 'C3' 'C4' 'CP5' 'CP6' 'CP1' 'CP2' 'P3' 'P4' 'PZ' 'P8' 'P7'
 'PO2' 'PO1' 'O2' 'O1' 'X' 'AF7' 'AF8' 'F5' 'F6' 'FT7' 'FT8' 'FPZ' 'FC4'
 'FC3' 'C6' 'C5' 'F2' 'F1' 'TP8' 'TP7' 'AFZ' 'CP3' 'CP4' 'P5' 'P6' 'C1'
 'C2' 'PO7' 'PO8' 'FCZ' 'POZ' 'OZ' 'P2' 'P1' 'CPZ' 'nd' 'Y']


In [0]:
def dfWithName(senPos):
  temp = pd.DataFrame(columns=senPos)
  for i in senPos:
    temp[i] = EEG_data.loc[EEG_data['sensor position'] == i]['sensor value'].to_numpy()
  temp['subject identifier'] = EEG_data.loc[EEG_data['sensor position'] == 'FP1']['subject identifier'].to_numpy()
  return temp

In [0]:
refacEegData = dfWithName(senPos)

In [128]:
refacEegData

,FP1,FP2,F7,F8,AF1,AF2,FZ,F4,F3,FC6,FC5,FC2,FC1,T8,T7,CZ,C3,C4,CP5,CP6,CP1,CP2,P3,P4,PZ,P8,P7,PO2,PO1,O2,O1,X,AF7,AF8,F5,F6,FT7,FT8,FPZ,FC4,FC3,C6,C5,F2,F1,TP8,TP7,AFZ,CP3,CP4,P5,P6,C1,C2,PO7,PO8,FCZ,POZ,OZ,P2,P1,CPZ,nd,Y,subject identifier
0,9.430,10.071,12.990,13.357,8.881,8.667,5.137,7.680,5.646,3.133,6.999,3.276,3.011,5.900,9.542,14.048,3.815,3.967,7.548,3.398,4.089,4.100,5.839,2.319,4.517,-1.038,11.047,3.815,6.877,2.350,8.962,10.284,12.410,7.945,9.511,7.690,12.410,9.644,9.481,3.794,3.133,3.296,3.316,5.890,5.391,2.319,8.118,7.721,5.971,4.720,8.403,0.865,2.635,4.415,11.414,0.356,2.614,4.924,5.300,1.882,4.333,4.374,9.023,11.149,a
1,6.500,7.629,11.037,9.939,4.974,4.761,2.207,4.262,3.204,3.133,5.534,1.811,1.546,6.877,11.007,13.072,2.838,4.456,6.571,4.862,6.042,6.053,7.304,3.784,6.470,-0.549,10.071,4.303,5.900,2.350,7.009,9.796,9.969,5.992,8.046,5.249,13.387,7.690,6.551,2.818,1.668,5.249,1.851,2.960,2.462,3.296,7.629,3.815,5.483,5.697,7.914,2.330,2.635,4.903,8.972,0.356,0.661,4.924,4.323,2.858,5.798,6.327,6.582,12.126,a
2,3.571,4.700,7.619,5.544,2.045,0.854,-0.722,1.333,1.251,4.110,5.534,0.346,0.570,5.900,10.030,12.095,2.350,3.967,4.618,5.351,6.531,6.053,7.304,4.272,6.958,0.427,8.118,4.791,5.412,2.350,4.567,8.820,8.016,4.038,5.605,4.272,12.410,3.296,3.621,2.330,1.180,6.226,1.363,0.519,0.020,3.296,6.165,0.397,4.995,4.720,7.426,2.330,2.635,4.415,6.531,0.844,-0.315,4.924,3.347,3.347,5.798,6.816,4.628,12.614,a
3,1.617,2.747,4.201,1.638,-0.397,-1.099,-2.187,-0.132,-0.214,5.086,6.999,-0.142,0.081,3.459,7.100,11.607,2.350,2.014,4.130,3.886,6.531,4.588,7.304,3.784,6.470,0.427,7.141,4.791,5.412,2.838,4.567,9.796,6.551,3.062,3.652,3.296,8.993,-1.587,1.668,2.330,1.180,4.272,2.828,-1.434,-1.444,1.831,5.188,-2.045,5.483,2.279,6.938,1.841,2.635,2.950,6.531,1.333,-0.804,4.435,3.835,2.858,5.310,6.327,4.628,12.614,a
4,1.617,3.235,4.201,1.638,-0.397,-0.610,-1.699,0.844,-1.190,5.086,8.464,-0.142,0.570,0.041,4.659,11.607,3.326,0.061,5.595,1.933,6.531,2.146,7.304,2.808,5.981,-0.061,7.629,3.815,6.388,3.815,7.009,12.238,6.551,3.062,3.652,3.296,5.575,-3.540,2.157,1.841,1.668,1.343,5.269,-0.946,-1.444,-0.610,5.188,-2.045,6.460,-0.163,7.914,1.353,3.123,0.997,7.996,1.821,-0.315,4.924,4.812,2.370,5.310,5.839,7.070,12.614,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119803,-85.022,-1.851,-4.222,0.722,-3.367,-0.692,0.682,2.594,-0.183,1.322,-1.699,1.943,1.170,3.916,-2.838,-8.911,-1.292,2.075,-4.751,0.936,-1.953,-2.065,-4.038,-1.302,0.285,-1.862,-4.313,-2.777,-6.215,-0.051,0.488,0.692,-3.123,-0.966,-2.452,2.818,-4.649,-0.397,-2.218,3.520,0.336,-1.160,-2.665,0.997,-0.397,-0.031,-0.641,-3.316,-8.535,-1.007,-3.215,-0.244,1.241,1.149,-0.102,11.627,1.465,0.539,-4.120,-10.518,-0.366,0.173,-6.561,-4.608,a
119804,-93.811,-2.828,-6.175,-2.207,-4.344,-2.645,-0.295,1.129,-1.648,-0.142,-2.675,1.455,0.682,-1.455,-6.256,-7.935,-0.315,1.587,-1.821,4.354,-1.465,-2.553,-4.527,-2.279,0.285,-3.815,-5.290,-3.265,-7.192,-1.027,0.000,-9.074,-3.611,-0.966,-4.405,2.330,-4.161,2.045,-3.194,3.520,0.336,-2.136,-2.177,0.997,-0.885,-5.890,-2.106,-4.293,-8.046,-1.984,-3.703,-2.686,1.729,1.149,-0.102,7.721,1.465,0.051,-4.120,-11.007,-1.343,0.173,-7.050,-9.979,a
119805,6.775,-4.293,-6.663,-2.696,-4.832,-3.133,-0.295,0.153,-2.136,-1.119,-3.164,0.478,0.682,-7.314,-8.209,-2.075,0.661,0.610,-1.821,0.936,-1.465,-3.530,-4.038,-3.255,0.285,-4.791,-5.290,-2.777,-7.680,-2.492,0.000,-15.910,-3.123,-0.478,-4.893,1.841,-3.184,3.021,-6.612,2.055,0.336,-1.160,-1.200,0.997,-0.885,-9.796,-5.035,-3.805,-7.070,-2.960,-2.726,-4.639,1.729,0.661,0.387,4.303,1.465,0.051,-3.143,-11.007,-1.343,-0.315,-5.585,-12.909,a
119806,91.736,-6.246,-5.686,-1.719,-4.832,-2.645,-0.783,-0.336,-1.648,-1.119,-3.164,0.478,0.682,-9.755,-7.721,5.249,0.661,3.052,-5.727,-1.017,-0.977,-3.530,-3.062,-3.743,0.773,-6.256,

In [0]:
refacEegData.to_csv('/content/refactoredEegData.csv')

In [0]:
eegData = pd.read_csv('/content/refactoredEegData.csv')

In [0]:
eegData.drop('Unnamed: 0',axis=1,inplace=True)

In [132]:
eegData

,FP1,FP2,F7,F8,AF1,AF2,FZ,F4,F3,FC6,FC5,FC2,FC1,T8,T7,CZ,C3,C4,CP5,CP6,CP1,CP2,P3,P4,PZ,P8,P7,PO2,PO1,O2,O1,X,AF7,AF8,F5,F6,FT7,FT8,FPZ,FC4,FC3,C6,C5,F2,F1,TP8,TP7,AFZ,CP3,CP4,P5,P6,C1,C2,PO7,PO8,FCZ,POZ,OZ,P2,P1,CPZ,nd,Y,subject identifier
0,9.430,10.071,12.990,13.357,8.881,8.667,5.137,7.680,5.646,3.133,6.999,3.276,3.011,5.900,9.542,14.048,3.815,3.967,7.548,3.398,4.089,4.100,5.839,2.319,4.517,-1.038,11.047,3.815,6.877,2.350,8.962,10.284,12.410,7.945,9.511,7.690,12.410,9.644,9.481,3.794,3.133,3.296,3.316,5.890,5.391,2.319,8.118,7.721,5.971,4.720,8.403,0.865,2.635,4.415,11.414,0.356,2.614,4.924,5.300,1.882,4.333,4.374,9.023,11.149,a
1,6.500,7.629,11.037,9.939,4.974,4.761,2.207,4.262,3.204,3.133,5.534,1.811,1.546,6.877,11.007,13.072,2.838,4.456,6.571,4.862,6.042,6.053,7.304,3.784,6.470,-0.549,10.071,4.303,5.900,2.350,7.009,9.796,9.969,5.992,8.046,5.249,13.387,7.690,6.551,2.818,1.668,5.249,1.851,2.960,2.462,3.296,7.629,3.815,5.483,5.697,7.914,2.330,2.635,4.903,8.972,0.356,0.661,4.924,4.323,2.858,5.798,6.327,6.582,12.126,a
2,3.571,4.700,7.619,5.544,2.045,0.854,-0.722,1.333,1.251,4.110,5.534,0.346,0.570,5.900,10.030,12.095,2.350,3.967,4.618,5.351,6.531,6.053,7.304,4.272,6.958,0.427,8.118,4.791,5.412,2.350,4.567,8.820,8.016,4.038,5.605,4.272,12.410,3.296,3.621,2.330,1.180,6.226,1.363,0.519,0.020,3.296,6.165,0.397,4.995,4.720,7.426,2.330,2.635,4.415,6.531,0.844,-0.315,4.924,3.347,3.347,5.798,6.816,4.628,12.614,a
3,1.617,2.747,4.201,1.638,-0.397,-1.099,-2.187,-0.132,-0.214,5.086,6.999,-0.142,0.081,3.459,7.100,11.607,2.350,2.014,4.130,3.886,6.531,4.588,7.304,3.784,6.470,0.427,7.141,4.791,5.412,2.838,4.567,9.796,6.551,3.062,3.652,3.296,8.993,-1.587,1.668,2.330,1.180,4.272,2.828,-1.434,-1.444,1.831,5.188,-2.045,5.483,2.279,6.938,1.841,2.635,2.950,6.531,1.333,-0.804,4.435,3.835,2.858,5.310,6.327,4.628,12.614,a
4,1.617,3.235,4.201,1.638,-0.397,-0.610,-1.699,0.844,-1.190,5.086,8.464,-0.142,0.570,0.041,4.659,11.607,3.326,0.061,5.595,1.933,6.531,2.146,7.304,2.808,5.981,-0.061,7.629,3.815,6.388,3.815,7.009,12.238,6.551,3.062,3.652,3.296,5.575,-3.540,2.157,1.841,1.668,1.343,5.269,-0.946,-1.444,-0.610,5.188,-2.045,6.460,-0.163,7.914,1.353,3.123,0.997,7.996,1.821,-0.315,4.924,4.812,2.370,5.310,5.839,7.070,12.614,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119803,-85.022,-1.851,-4.222,0.722,-3.367,-0.692,0.682,2.594,-0.183,1.322,-1.699,1.943,1.170,3.916,-2.838,-8.911,-1.292,2.075,-4.751,0.936,-1.953,-2.065,-4.038,-1.302,0.285,-1.862,-4.313,-2.777,-6.215,-0.051,0.488,0.692,-3.123,-0.966,-2.452,2.818,-4.649,-0.397,-2.218,3.520,0.336,-1.160,-2.665,0.997,-0.397,-0.031,-0.641,-3.316,-8.535,-1.007,-3.215,-0.244,1.241,1.149,-0.102,11.627,1.465,0.539,-4.120,-10.518,-0.366,0.173,-6.561,-4.608,a
119804,-93.811,-2.828,-6.175,-2.207,-4.344,-2.645,-0.295,1.129,-1.648,-0.142,-2.675,1.455,0.682,-1.455,-6.256,-7.935,-0.315,1.587,-1.821,4.354,-1.465,-2.553,-4.527,-2.279,0.285,-3.815,-5.290,-3.265,-7.192,-1.027,0.000,-9.074,-3.611,-0.966,-4.405,2.330,-4.161,2.045,-3.194,3.520,0.336,-2.136,-2.177,0.997,-0.885,-5.890,-2.106,-4.293,-8.046,-1.984,-3.703,-2.686,1.729,1.149,-0.102,7.721,1.465,0.051,-4.120,-11.007,-1.343,0.173,-7.050,-9.979,a
119805,6.775,-4.293,-6.663,-2.696,-4.832,-3.133,-0.295,0.153,-2.136,-1.119,-3.164,0.478,0.682,-7.314,-8.209,-2.075,0.661,0.610,-1.821,0.936,-1.465,-3.530,-4.038,-3.255,0.285,-4.791,-5.290,-2.777,-7.680,-2.492,0.000,-15.910,-3.123,-0.478,-4.893,1.841,-3.184,3.021,-6.612,2.055,0.336,-1.160,-1.200,0.997,-0.885,-9.796,-5.035,-3.805,-7.070,-2.960,-2.726,-4.639,1.729,0.661,0.387,4.303,1.465,0.051,-3.143,-11.007,-1.343,-0.315,-5.585,-12.909,a
119806,91.736,-6.246,-5.686,-1.719,-4.832,-2.645,-0.783,-0.336,-1.648,-1.119,-3.164,0.478,0.682,-9.755,-7.721,5.249,0.661,3.052,-5.727,-1.017,-0.977,-3.530,-3.062,-3.743,0.773,-6.256,

In [0]:
X = eegData.drop('subject identifier',axis=1)
y  = eegData['subject identifier']

In [0]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [135]:
label_encoder = LabelEncoder()
label_encoder.fit(np.array(y))
integer_encoded_y = label_encoder.transform(np.array(y))
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded_y = integer_encoded_y.reshape(len(integer_encoded_y), 1)
target = onehot_encoder.fit_transform(integer_encoded_y)
target

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

In [0]:
# Simple Deep Learning Model

dlModel = Sequential()
dlModel.add(Dense(85, input_dim=64, activation='relu'))
dlModel.add(BatchNormalization())
dlModel.add(Dropout(0.25))

dlModel.add(Dense(85 ,activation='relu'))
dlModel.add(BatchNormalization())
dlModel.add(Dropout(0.25))

dlModel.add(Dense(85 ,activation='relu'))
dlModel.add(BatchNormalization())
dlModel.add(Dropout(0.25))

dlModel.add(Dense(2, activation='sigmoid'))
# Compile dlModel
dlModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [138]:
X_train.shape,X_test.shape

((95846, 64), (23962, 64))

In [139]:
r = dlModel.fit(X_train,y_train,batch_size=128,epochs=20,validation_split=0.2)
accr = dlModel.evaluate(X_test,y_test)

Train on 76676 samples, validate on 19170 samples
Epoch 1/20
76676/76676 [==============================] - 3s 40us/step - loss: 0.5299 - accuracy: 0.7273 - val_loss: 0.3150 - val_accuracy: 0.8625
Epoch 2/20
76676/76676 [==============================] - 3s 34us/step - loss: 0.3268 - accuracy: 0.8564 - val_loss: 0.2025 - val_accuracy: 0.9227
Epoch 3/20
76676/76676 [==============================] - 2s 32us/step - loss: 0.2598 - accuracy: 0.8898 - val_loss: 0.1491 - val_accuracy: 0.9456
Epoch 4/20
76676/76676 [==============================] - 2s 32us/step - loss: 0.2141 - accuracy: 0.9122 - val_loss: 0.1431 - val_accuracy: 0.9439
Epoch 5/20
76676/76676 [==============================] - 3s 34us/step - loss: 0.2001 - accuracy: 0.9176 - val_loss: 0.1075 - val_accuracy: 0.9613
Epoch 6/20
76676/76676 [==============================] - 3s 33us/step - loss: 0.1737 - accuracy: 0.9305 - val_loss: 0.0912 - val_accuracy: 0.9671
Epoch 7/20
76676/76676 [==============================] - 3s 34us/st

In [140]:
dlModel_json = dlModel.to_json()
with open("/content/dlModel.json", "w") as json_file:
    json_file.write(dlModel_json)
# serialize weights to HDF5
dlModel.save_weights("/content/dlModel.h5")
print("Saved dlModel to disk")

Saved dlModel to disk


array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [152]:
inp = np.matrix([-14.76,-15.839,-10.315,-9.898,-12.767,-12.431,-8.87,-8.077,-10.457,-7.874,-9.521,
-3.682,-6.429,-4.323,-9.969,-10.284,-6.449,-1.404,-7.629,-2.218,-2.035,4.415,-3.906,-1.434,-1.322,-6.348,-5.768,-2.085,-4.496,-4.059,-6.307,-14.801,-16.907,-19.602,-12.258,-9.857,-10.834,-8.901,-13.784,-6.022,-8.209,-4.679,-8.87,-9.878,-10.02,-5.544,-7.833,-12.594,-5.076,-0.295,-4.954,-1.719,-2.502,0.468,-9.918,-5.809,-4.506,-2.523,-4.761,1.18,-1.139,1.353,-6.388,-10.366])

inp

matrix([[-14.76 , -15.839, -10.315,  -9.898, -12.767, -12.431,  -8.87 ,
          -8.077, -10.457,  -7.874,  -9.521,  -3.682,  -6.429,  -4.323,
          -9.969, -10.284,  -6.449,  -1.404,  -7.629,  -2.218,  -2.035,
           4.415,  -3.906,  -1.434,  -1.322,  -6.348,  -5.768,  -2.085,
          -4.496,  -4.059,  -6.307, -14.801, -16.907, -19.602, -12.258,
          -9.857, -10.834,  -8.901, -13.784,  -6.022,  -8.209,  -4.679,
          -8.87 ,  -9.878, -10.02 ,  -5.544,  -7.833, -12.594,  -5.076,
          -0.295,  -4.954,  -1.719,  -2.502,   0.468,  -9.918,  -5.809,
          -4.506,  -2.523,  -4.761,   1.18 ,  -1.139,   1.353,  -6.388,
         -10.366]])

In [0]:
prediction = dlModel.predict(inp)

In [171]:
predvalue= (prediction[0] > 0.5) & (prediction[0] < 1 )
if predvalue[0]== True:
  print("alcoholic")
else:
  print("non-alcoholic")

alcoholic
